# ETL Pipeline Demo (Schema-on-Read)

This notebook showcases the end-to-end Extract → Transform → Load flow used in the project. It reads a Parquet file from MinIO (S3-compatible), converts each record into a canonical JSON row with provenance, and loads the result into a ClickHouse table over HTTP. The goal is to demonstrate transparency and repeatability with minimal moving parts.

Outline:
- Environment and configuration
- Spark session (S3A → MinIO)
- Extract (read Parquet as-is)
- Transform (timestamp, data_tag, datavalue_json)
- Load (HTTP INSERT into ClickHouse)
- Quick verification


In [1]:
# Environment & libraries
import os, sys, math, time
from urllib import request as urlrequest
from urllib.parse import quote
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Load .env at project root (one level up from this notebook)
project_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
env_path = os.path.join(project_root, '.env')
load_dotenv(env_path)

# MinIO (S3A)
MINIO_URL = os.environ.get('MINIO_ENDPOINT_URL')
MINIO_KEY = os.environ.get('MINIO_ACCESS_KEY')
MINIO_SECRET = os.environ.get('MINIO_SECRET_KEY')
MINIO_BUCKET = os.environ.get('MINIO_BUCKET_NAME')
MINIO_FILE = os.environ.get('MINIO_TEST_FILE_PATH')  # single path for this demo

if not MINIO_URL or not MINIO_BUCKET or not MINIO_FILE:
    raise RuntimeError('Missing MINIO_ENDPOINT_URL / MINIO_BUCKET_NAME / MINIO_TEST_FILE_PATH in .env')

if MINIO_URL.startswith('http://'):
    MINIO_HOST_PORT = MINIO_URL[7:]
elif MINIO_URL.startswith('https://'):
    MINIO_HOST_PORT = MINIO_URL[8:]
else:
    MINIO_HOST_PORT = MINIO_URL

# ClickHouse
CH_HOST = os.environ.get('CLICKHOUSE_HOST')
CH_PORT = os.environ.get('CLICKHOUSE_PORT', '8787')
CH_USER = os.environ.get('CLICKHOUSE_USER', 'admin')
CH_PASS = os.environ.get('CLICKHOUSE_PASSWORD', '')
CH_DB   = os.environ.get('CLICKHOUSE_DATABASE', 'default')
CH_TABLE= os.environ.get('CLICKHOUSE_TABLE', 'skz_pipeline_data')

# Windows Hadoop helpers (winutils)
hadoop_home = os.path.join(project_root, 'hadoop')
hadoop_bin = os.path.join(hadoop_home, 'bin')
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['hadoop.home.dir'] = hadoop_home
if os.path.isdir(hadoop_bin):
    os.environ['PATH'] = os.environ.get('PATH', '') + os.pathsep + hadoop_bin

# Helper to build ClickHouse HTTP inserts
def ch_http_insert_tsv(rows):
    if not rows:
        return
    lines = []
    for ts, tag, json_value in rows:
        try:
            ts = ts.replace(microsecond=0)
        except Exception:
            pass
        ts_str = ts.strftime('%Y-%m-%d %H:%M:%S')
        lines.append(f"{ts_str}\t{str(tag).replace('\t','\\t').replace('\n','\\n')}\t{str(json_value).replace('\t','\\t').replace('\n','\\n')}")
    tsv_body = "\n".join(lines)

    # Preferred: SQL in query=, data in body (FORMAT TSV)
    insert_q = f"INSERT INTO {CH_DB}.{CH_TABLE} (timestamp, data_tag, datavalue_json) FORMAT TSV"
    url_q = f"http://{CH_HOST}:{CH_PORT}/?database={CH_DB}&user={CH_USER}&password={CH_PASS}&query={quote(insert_q)}"
    req = urlrequest.Request(url_q, data=tsv_body.encode('utf-8'), method='POST')
    req.add_header('Content-Type', 'text/plain; charset=utf-8')
    try:
        with urlrequest.urlopen(req) as resp:
            _ = resp.read()
            return
    except Exception:
        # Fallback: SQL completo no corpo
        insert_sql = f"INSERT INTO {CH_DB}.{CH_TABLE} (timestamp, data_tag, datavalue_json) FORMAT TSV\n" + tsv_body
        url_body = f"http://{CH_HOST}:{CH_PORT}/?database={CH_DB}&user={CH_USER}&password={CH_PASS}"
        req2 = urlrequest.Request(url_body, data=insert_sql.encode('utf-8'), method='POST')
        req2.add_header('Content-Type', 'text/plain; charset=utf-8')
        with urlrequest.urlopen(req2) as resp2:
            _ = resp2.read()

# Spark session (S3A → MinIO)
spark = (
    SparkSession.builder
    .appName('ETL_Demo_Notebook')
    .config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.367')
    .config('spark.hadoop.fs.s3a.endpoint', MINIO_HOST_PORT)
    .config('spark.hadoop.fs.s3a.access.key', MINIO_KEY)
    .config('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET)
    .config('spark.hadoop.fs.s3a.aws.credentials.provider','org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    .config('spark.hadoop.fs.s3a.path.style.access','true')
    .config('spark.hadoop.fs.s3a.connection.ssl.enabled','false')
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
    .getOrCreate()
)

print('Environment ready.')


Environment ready.


In [3]:
# Harden S3A timeouts (ms) to avoid '60s/24h' parse issues
conf = spark._jsc.hadoopConfiguration()
conf.set("fs.s3a.connection.timeout", "60000")
conf.set("fs.s3a.connection.establish.timeout", "5000")
conf.set("fs.s3a.idle.connection.timeout", "300000")
conf.set("fs.s3a.threads.keepalivetime", "60000")
conf.set("fs.s3a.threads.shutdown.timeout", "60000")
conf.set("fs.s3a.multipart.purge", "true")
conf.set("fs.s3a.multipart.purge.age", "86400000")
print("S3A timeouts configured.")


S3A timeouts configured.


## Extract

Read the Parquet file directly from MinIO using the S3A connector. The intent is to validate connectivity and observe the source schema as-is (no mutation here).


In [4]:
s3_path = f"s3a://{MINIO_BUCKET}/{MINIO_FILE}"
print('Reading from:', s3_path)

raw_df = spark.read.parquet(s3_path)
print('Source schema:')
raw_df.printSchema()

print('Sample rows:')
raw_df.show(5, truncate=False)


Reading from: s3a://data/batch_001/ctccusto/ctccusto.parquet
Source schema:
root
 |-- codi_emp: integer (nullable = true)
 |-- i_ccusto: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipo: string (nullable = true)

Sample rows:
+--------+--------+-----------------+--------------+
|codi_emp|i_ccusto|descricao        |tipo          |
+--------+--------+-----------------+--------------+
|5000    |500001  |Serviços AM      |OPERACIONAL   |
|5000    |500002  |Administrativo AM|ADMINISTRATIVO|
|5000    |500003  |Compras AM       |OPERACIONAL   |
|5000    |500004  |Jurídico AM      |OPERACIONAL   |
|5001    |500101  |Operações CE     |OPERACIONAL   |
+--------+--------+-----------------+--------------+
only showing top 5 rows


## Transform

Build the canonical landing shape (schema-on-read):
- `timestamp`: processing time
- `data_tag`: batch/file provenance (derived from the MinIO path)
- `datavalue_json`: full source row serialized as JSON


In [5]:
data_tag = MINIO_FILE.replace('.parquet','').replace('/','_')
all_cols = raw_df.columns

transformed_df = (
    raw_df
    .withColumn('datavalue_json', F.to_json(F.struct(*all_cols)))
    .withColumn('data_tag', F.lit(data_tag))
    .withColumn('timestamp', F.current_timestamp())
    .select('timestamp','data_tag','datavalue_json')
)

print('Landing schema:')
transformed_df.printSchema()
print('Sample JSON rows:')
transformed_df.show(5, truncate=False)


Landing schema:
root
 |-- timestamp: timestamp (nullable = false)
 |-- data_tag: string (nullable = false)
 |-- datavalue_json: string (nullable = true)

Sample JSON rows:
+--------------------------+---------------------------+-------------------------------------------------------------------------------------------+
|timestamp                 |data_tag                   |datavalue_json                                                                             |
+--------------------------+---------------------------+-------------------------------------------------------------------------------------------+
|2025-11-07 19:31:40.297242|batch_001_ctccusto_ctccusto|{"codi_emp":5000,"i_ccusto":500001,"descricao":"Serviços AM","tipo":"OPERACIONAL"}         |
|2025-11-07 19:31:40.297242|batch_001_ctccusto_ctccusto|{"codi_emp":5000,"i_ccusto":500002,"descricao":"Administrativo AM","tipo":"ADMINISTRATIVO"}|
|2025-11-07 19:31:40.297242|batch_001_ctccusto_ctccusto|{"codi_emp":5000,"i_ccusto"

## Load

Insert the canonical rows into ClickHouse using the HTTP interface. The loader writes in batches and reports progress with an ETA.


In [6]:
total_rows = transformed_df.count()
print(f'Rows to load: {total_rows:,} | target={CH_DB}.{CH_TABLE}')

batch = []
batch_size = 1000
processed = 0
start_ts = time.time()

def fmt(t):
    m, s = divmod(int(t), 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

for row in transformed_df.toLocalIterator():
    ts = row['timestamp']
    try:
        ts = ts.replace(microsecond=0)
    except Exception:
        pass
    batch.append((ts, row['data_tag'], row['datavalue_json']))
    if len(batch) >= batch_size:
        ch_http_insert_tsv(batch)
        processed += len(batch)
        batch.clear()
        elapsed = time.time() - start_ts
        rate = processed / elapsed if elapsed > 0 else 0
        eta = (total_rows - processed) / rate if rate > 0 else 0
        pct = (processed / total_rows * 100.0) if total_rows > 0 else 0.0
        bar_w = 30
        filled = int(bar_w * pct / 100.0)
        bar = '#' * filled + '-' * (bar_w - filled)
        print(f"\r[LOAD] {processed:,}/{total_rows:,} ({pct:5.1f}%) [{bar}] elapsed {fmt(elapsed)} eta {fmt(eta)} ", end='')

if batch:
    ch_http_insert_tsv(batch)
    processed += len(batch)
    elapsed = time.time() - start_ts
    print(f"\r[LOAD] {processed:,}/{total_rows:,} (100.0%) [##############################] elapsed {fmt(elapsed)} eta 00:00:00      ")

print('\nLoad finished.')


Rows to load: 800 | target=DELIVERABLE_SPRINT_2.G01_pipeline_results
[LOAD] 800/800 (100.0%) [##############################] elapsed 00:00:00 eta 00:00:00      

Load finished.


## Quick verification

Run a quick count and preview a few rows from the target table to confirm the load.


In [7]:
# COUNT(*)
count_url = f"http://{CH_HOST}:{CH_PORT}/?query=SELECT%20count()%20FROM%20{CH_DB}.{CH_TABLE}&user={CH_USER}&password={CH_PASS}"
with urlrequest.urlopen(count_url) as resp:
    body = resp.read().decode('utf-8').strip()
print('Row count:', body)

# Tail sample
tail_q = quote(f"SELECT timestamp, data_tag, substring(datavalue_json,1,120) FROM {CH_DB}.{CH_TABLE} ORDER BY timestamp DESC LIMIT 5")
tail_url = f"http://{CH_HOST}:{CH_PORT}/?query={tail_q}&user={CH_USER}&password={CH_PASS}"
with urlrequest.urlopen(tail_url) as resp:
    tail = resp.read().decode('utf-8').strip()
print('\nRecent rows:\n', tail)


Row count: 18718

Recent rows:
 2025-11-07 19:31:47	batch_001_ctccusto_ctccusto	{"codi_emp":5132,"i_ccusto":513204,"descricao":"Administrativo AM","tipo":"ADMINISTRATIVO"}
2025-11-07 19:31:47	batch_001_ctccusto_ctccusto	{"codi_emp":5133,"i_ccusto":513301,"descricao":"Jurídico RS","tipo":"OPERACIONAL"}
2025-11-07 19:31:47	batch_001_ctccusto_ctccusto	{"codi_emp":5134,"i_ccusto":513402,"descricao":"Serviços MT","tipo":"OPERACIONAL"}
2025-11-07 19:31:47	batch_001_ctccusto_ctccusto	{"codi_emp":5133,"i_ccusto":513303,"descricao":"Administrativo RS","tipo":"ADMINISTRATIVO"}
2025-11-07 19:31:47	batch_001_ctccusto_ctccusto	{"codi_emp":5133,"i_ccusto":513302,"descricao":"Manutenção RS","tipo":"OPERACIONAL"}


## Integrated ETL Pipeline (run-to-completion)

This cell loops through all discovered batches (`batch_*/`) and all Parquet files within each batch, applying the same Extract → Transform → Load pattern until completion. Progress and ETA are printed inline.


In [ ]:
import boto3
from botocore.client import Config

# Parameters
FRACTION = 1.0  # set to 0.01 for 1% sampling if needed
BATCH_PRINT_LIMIT = 10

s3 = boto3.client(
    's3',
    endpoint_url=MINIO_URL,
    aws_access_key_id=MINIO_KEY,
    aws_secret_access_key=MINIO_SECRET,
    use_ssl=MINIO_URL.startswith('https'),
    config=Config(signature_version='s3v4')
)

# Helpers

def list_with_retry(**params):
    delay = 0.5
    for attempt in range(5):
        try:
            return s3.list_objects_v2(**params)
        except Exception:
            if attempt == 4:
                raise
            time.sleep(delay)
            delay *= 2
    return {}

def discover_batches():
    batch_ids = []
    token = None
    while True:
        params = dict(Bucket=MINIO_BUCKET, Prefix='batch_', Delimiter='/', MaxKeys=1000)
        if token:
            params['ContinuationToken'] = token
        resp = list_with_retry(**params)
        for cp in resp.get('CommonPrefixes', []):
            p = cp.get('Prefix','')  # e.g., 'batch_001/'
            try:
                num = p.strip('/').split('_',1)[1]
                if num.isdigit():
                    batch_ids.append(str(int(num)))
            except Exception:
                pass
        if resp.get('IsTruncated'):
            token = resp.get('NextContinuationToken')
        else:
            break
    return sorted(set(batch_ids), key=lambda x: int(x))

def list_parquet_keys(batch_ids):
    keys = []
    for bid in batch_ids:
        prefix = f"batch_{int(bid):03d}/"
        token = None
        while True:
            params = dict(Bucket=MINIO_BUCKET, Prefix=prefix, MaxKeys=1000)
            if token:
                params['ContinuationToken'] = token
            resp = list_with_retry(**params)
            for obj in resp.get('Contents', []):
                k = obj['Key']
                if k.endswith('.parquet'):
                    keys.append(k)
            if resp.get('IsTruncated'):
                token = resp.get('NextContinuationToken')
            else:
                break
    return keys

# Discovery
print('===== DISCOVERY =====')
all_batches = discover_batches()
print(f"Batches found: {len(all_batches)} | first {BATCH_PRINT_LIMIT} = {all_batches[:BATCH_PRINT_LIMIT]}")
all_keys = list_parquet_keys(all_batches)
print(f"Parquet files to process: {len(all_keys)}")

# Run
file_idx = 0
start_all = time.time()

for key in all_keys:
    file_idx += 1
    data_tag = key.replace('.parquet','').replace('/','_')
    s3_path = f"s3a://{MINIO_BUCKET}/{key}"

    print(f"\n===== FILE {file_idx}/{len(all_keys)} =====")
    print('[EXTRACT]', s3_path)
    df = spark.read.parquet(s3_path)

    if FRACTION < 1.0:
        df = df.sample(withReplacement=False, fraction=FRACTION, seed=42)
        print(f"Sampling {FRACTION*100:.1f}% of rows")

    # Transform
    cols = df.columns
    landed = (
        df
        .withColumn('datavalue_json', F.to_json(F.struct(*cols)))
        .withColumn('data_tag', F.lit(data_tag))
        .withColumn('timestamp', F.current_timestamp())
        .select('timestamp','data_tag','datavalue_json')
    )

    total_rows = landed.count()
    print(f"[PLAN] data_tag={data_tag} | rows={total_rows:,}")

    # Load with progress
    processed = 0
    batch = []
    batch_size = 1000
    start_ts = time.time()

    def fmt(t):
        m,s = divmod(int(t),60)
        h,m = divmod(m,60)
        return f"{h:02d}:{m:02d}:{s:02d}"

    for r in landed.toLocalIterator():
        ts = r['timestamp']
        try:
            ts = ts.replace(microsecond=0)
        except Exception:
            pass
        batch.append((ts, r['data_tag'], r['datavalue_json']))
        if len(batch) >= batch_size:
            ch_http_insert_tsv(batch)
            processed += len(batch)
            batch.clear()
            elapsed = time.time() - start_ts
            rate = processed / elapsed if elapsed>0 else 0
            eta = (total_rows-processed)/rate if rate>0 else 0
            pct = processed/total_rows*100 if total_rows>0 else 0
            bar_w=30; filled=int(bar_w*pct/100); bar='#'*filled+'-'*(bar_w-filled)
            print(f"\r[LOAD] {processed:,}/{total_rows:,} ({pct:5.1f}%) [{bar}] elapsed {fmt(elapsed)} eta {fmt(eta)} ", end='')

    if batch:
        ch_http_insert_tsv(batch)
        processed += len(batch)
        elapsed = time.time() - start_ts
        print(f"\r[LOAD] {processed:,}/{total_rows:,} (100.0%) [##############################] elapsed {fmt(elapsed)} eta 00:00:00      ")

print(f"\n===== DONE (ALL) in {fmt(time.time()-start_all)} =====")
